In [47]:
import numpy as np

In [1]:
file_path = r'D:\Atomsk\las_cif\las2o6-c\las2o6-c-232.xyz'

In [3]:
# 参数


# 晶体

In [6]:
# 读取xyz文件并按照给定条件筛选坐标
def read_xyz_file_with_conditions(filename):
    coordinates = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for line in lines[2:]:  # 从第三行开始读取坐标数据
            parts = line.split()
            if len(parts) >= 4:
                atom = parts[0]
                #  转换一下单位 
                x = float(parts[1])
                y = float(parts[2])
                z = float(parts[3])
                coordinates.append((atom, x, y, z))
    return coordinates

# 使用示例
filename = file_path  # 替换为你的xyz文件名
cry_xyz = read_xyz_file_with_conditions(filename)

In [8]:
# 原始坐标列表
coordinates = cry_xyz
# 晶体中原子的计数
num_cry_dic = create_dic = {'O':0,'Si':0,'Al':0,'Li':0}
# 平移的距离
a, b, c = 0, 0, 0.0  # 替换为你需要的平移值
# 初始化最大最小值
max_x = max_y = max_z = float('-inf')
min_x = min_y = min_z = float('inf')

new_cry_xyz = []

# 平移坐标并寻找最大最小值
for atom, x, y, z in coordinates:
    # 对每个坐标平移
    new_x = x + a
    new_y = y + b
    new_z = z + c
    new_cry_xyz.append([atom,new_x,new_y,new_z])
    # 对晶体在的原子计数
    num_cry_dic[atom] += 1
    # 更新最大值
    if new_x > max_x:
        max_x = new_x
    if new_y > max_y:
        max_y = new_y
    if new_z > max_z:
        max_z = new_z
    
    # 更新最小值
    if new_x < min_x:
        min_x = new_x
    if new_y < min_y:
        min_y = new_y
    if new_z < min_z:
        min_z = new_z

# 输出平移后所有原子中最大的和最小的 x, y, z 值
print("晶体的初始xyz范围：")
print(f"x 值: {min_x} -- {max_x}")
print(f"y 值: {min_y} -- {max_y}")
print(f"z 值: {min_z} -- {max_z}")
print("晶体的原子数：")
print(len(cry_xyz))
print(num_cry_dic)

晶体的初始xyz范围：
x 值: 0.162764087 -- 16.7821414668
y 值: -9.1729866171 -- 21.47911941
z 值: 0.3297567093 -- 9.5397223282
晶体的原子数：
480
{'O': 288, 'Si': 96, 'Al': 48, 'Li': 48}


In [10]:
charge_dic = {'O':-1.2,'Si':2.4,'Al':1.8,'Li':0.6}

def get_charge(atom_list):
    sum_charge = 0
    for i_atom in atom_list:
        temp_type = i_atom[0]
        sum_charge += charge_dic[temp_type]
    print('总电荷数： {}'.format(round(sum_charge,1)))
# 
get_charge(new_cry_xyz)

总电荷数： 0.0


In [12]:
num_cry_dic

{'O': 288, 'Si': 96, 'Al': 48, 'Li': 48}

# 筛选原子

In [15]:
# 原来的a b c
a = 8.4725
b = 9.6391  # Y上
c = 5.2751

In [83]:
# 找到一个周期性的边界  然后一边删除 一边扩胞
low_value, high_value =  -1.55704,17.72107 
len_value = high_value- low_value
print(len_value,b*2)

19.27811 19.2782


In [31]:
new_cry_xyz[0]

['Al', 7.6695695559, -9.1729866171, 1.2336848797]

In [33]:
new_list = []
for i_atom in new_cry_xyz:
    # 确定下比较的单位
    temp_value = i_atom[2]
    #print(i_atom)
    # 【区间是左闭右开）  要大于等于1.5 and 小于3.5
    if low_value<=temp_value<high_value:
        new_list.append(i_atom)
get_charge(new_list)

总电荷数： 0.0


# 当遇到xyz小于0的，先全部给它加上 

In [49]:
def move_to_0(atom_list):
    # 先遍历一边 然后得出xyz的范围
    # 原始坐标列表
    coordinates = atom_list
    # 初始化最大最小值
    max_x = max_y = max_z = float('-inf')
    min_x = min_y = min_z = float('inf')
    # 平移坐标并寻找最大最小值
    for atom, new_x, new_y, new_z in coordinates:
        # 更新最大值
        if new_x > max_x:
            max_x = new_x
        if new_y > max_y:
            max_y = new_y
        if new_z > max_z:
            max_z = new_z
        # 更新最小值
        if new_x < min_x:
            min_x = new_x
        if new_y < min_y:
            min_y = new_y
        if new_z < min_z:
            min_z = new_z
    # 找到最值了 然后平移
    move_x,move_y,move_z = -min_x,-min_y,-min_z
    # 原来数据
    print("原来晶体的xyz范围：")
    print(f"x 值: {min_x} -- {max_x}")
    print(f"y 值: {min_y} -- {max_y}")
    print(f"z 值: {min_z} -- {max_z}")
    # 存储所有新数据
    temp_all_list = []
    # 新的范围 
      # 初始化最大最小值
    max_x1 = max_y1 = max_z1 = float('-inf')
    min_x1 = min_y1 = min_z1 = float('inf')
    for atom, x, y, z in coordinates:
        x1,y1,z1 = x+move_x,y+move_y,z+move_z
        temp_all_list.append([atom,x1,y1,z1])
        ###
        # 更新最大值
        if x1 > max_x1:
            max_x1 = x1
        if y1 > max_y1:
            max_y1 = y1
        if z1 > max_z1:
            max_z1 = z1
        # 更新最小值
        if x1 < min_x1:
            min_x1 = x1
        if y1 < min_y1:
            min_y1 = x1
        if z1 < min_z1:
            min_z1 = x1
    
    # 输出平移后所有原子中最大的和最小的 x, y, z 值
    print("修正过后的晶体的xyz范围：")
    print(f"x 值: {min_x1} -- {max_x1}")
    print(f"y 值: {min_y1} -- {max_y1}")
    print(f"z 值: {min_z1} -- {max_z1}")
# 处理对象
    return temp_all_list
final_list = move_to_0(new_list)
print(get_charge(final_list))
print(f'原子数：{len(final_list)}')
print(f'切割的边长： {len_value}')

原来晶体的xyz范围：
x 值: 0.162764087 -- 16.7821414668
y 值: -1.5452816712 -- 17.7210692661
z 值: 0.3297567093 -- 9.5397223282
修正过后的晶体的xyz范围：
x 值: 0.0 -- 16.6193773798
y 值: 0.0 -- 19.2663509373
z 值: 0.0 -- 9.2099656189
总电荷数： 0.0
None
原子数：320
切割的边长： 19.27811


In [55]:
# 原有abc
a = 8.4725
b = 9.6391
c = 4.9347
#########
len_x,len_y,len_z = a*2,b*2,c*2
box_size = [len_x,len_y,len_z]
print(box_size)

[16.945, 19.2782, 9.8694]


In [59]:
count=1
for atom, x, y, z in  final_list:
    #print(f'{atom}{count}  {atom}  {x}  {y}  {z} 0.00000  Uiso   1.00')
    count+=1

In [61]:
5.2750*np.sin(np.deg2rad(69.3041183826))

4.934601274638562

In [72]:
temp_list = final_list
#
copy_list = []
#  复制晶胞  4,3,4,
copy_x,copy_y,copy_z = 4,3,6
# 生成新数据
for i_x in range(copy_x):
    for i_y in range(copy_y):
        for i_z in range(copy_z):
            # 然后遍历原子
            dx,dy,dz = i_x*len_x,i_y*len_y,i_z*len_z
            for atom,x,y,z in temp_list:
                temp_atom = [atom,x+dx,y+dy,z+dz]
                copy_list.append(temp_atom)
print('New boxsize:')
print(f'{len_x*copy_x}  {len_y*copy_y}  {len_z*copy_z}')
print(len(copy_list))

New boxsize:
67.78  57.834599999999995  59.21640000000001
23040


In [74]:
5760/720

8.0

In [77]:
"""
#LAMMPS data file transformed from *.xyz file
9213 atoms
4 atom types
    0.000000    46.100000 xlo xhi
    0.000000    46.100000 ylo yhi
    0.000000    46.100000 zlo zhi
    0.000000     0.000000     0.000000 xy xz yz

Atoms

      1 1  0.00    36.678737     2.066764    41.325880
      2 1  0.00     1.221441     3.114431     0.279197
      3 1  0.00    32.759957    19.543366     2.110140
      4 1  0.00    17.250993     4.961661     8.228568

"""

'\n#LAMMPS data file transformed from *.xyz file\n9213 atoms\n4 atom types\n    0.000000    46.100000 xlo xhi\n    0.000000    46.100000 ylo yhi\n    0.000000    46.100000 zlo zhi\n    0.000000     0.000000     0.000000 xy xz yz\n\nAtoms\n\n      1 1  0.00    36.678737     2.066764    41.325880\n      2 1  0.00     1.221441     3.114431     0.279197\n      3 1  0.00    32.759957    19.543366     2.110140\n      4 1  0.00    17.250993     4.961661     8.228568\n\n'

In [79]:

ele_dic = {'O':1,'Si':2,'Al':3,'Li':4}
ele_dic1 = {'O':'O','Si':'Si','Al':'Al','Li':'Li'}
# 将坐标写回到xyz文件中
def write_xyz_file(filename, coordinates):
    with open(filename, 'w') as f:
        f.write(f"{len(coordinates)}\n")
        f.write("Atoms\n")
        for atom, x, y, z in coordinates:
            f.write(f"{ele_dic1[atom]}    {x:.6f}    {y:.6f}    {z:.6f}\n")

##########
write_list = copy_list
save_path = r'D:\Atomsk\las_cif\las2o6-c\las2o6_c_cube_copy_si.xyz'
save_path_0 = r'D:\Atomsk\las_cif\las2o6-c\las2o6_c_cube0.xyz'

path = save_path_0
############

# 使用示例
write_xyz_file(path, write_list)
print(f"Filtered coordinates saved to {save_path}")


Filtered coordinates saved to D:\Atomsk\las_cif\las2o6-c\las2o6_c_cube_copy_si.xyz


In [84]:

ele_dic = {'O':1,'Si':2,'Al':3,'Li':4}
# 将坐标写回到dat文件中

def write_dat_file(filename, coordinates):
    with open(filename, 'w') as f:
        f.write('# Translate data using python code by Yajiao Zhang.\n')
        f.write(f"{len(coordinates)} atoms \n")
        f.write("4 atom types\n")
        f.write(f"    0.000000    {max_x:.6f}  xlo xhi\n")
        f.write(f"    0.000000    {max_y:.6f}  ylo yhi\n")
        f.write(f"    0.000000    {max_z:.6f}  zlo zhi\n")
        f.write('    0.000000     0.000000     0.000000 xy xz yz\n')
        f.write('\n')
        f.write("Atoms\n\n")
        count_num = 1
        for atom, x, y, z in coordinates:
            
            f.write(f"    {count_num}  {ele_dic[atom]}  0.00   {x:.6f}    {y:.6f}    {z:.6f}\n")
            count_num+=1
##########
write_list = final_list
save_path = r'D:\Atomsk\las_cif\las2o6_cube.xyz'
############
# 使用示例
write_dat_file(save_path, write_list)
print(f"Filtered coordinates saved to {save_path}")


Filtered coordinates saved to D:\Atomsk\las_cif\las2o6_cube.xyz


In [24]:
# 将lmp文件写进las_112.region
def write_xyz_file(filename, coordinates):
    with open(filename, 'w') as f:
        f.write("region box_region block {} {} {} {} {} {}\n\n".format(0, len_box, 0, len_box, 0, len_box))
        f.write("region cry_region block {} {} {} {} {} {}\n\n".format(low_x, high_x, low_y, high_y,low_z, high_z))
        f.write("group all_atoms region box_region\n\n")
        f.write("group cry_group region cry_region\n\n")
        f.write("group glass_group subtract all_atoms cry_group\n")
# 使用示例
save_path = save_region  # 设置保存文件的路径和文件名
write_xyz_file(save_path, all_xyz)
print(f"Filtered coordinates saved to {save_path}")


Filtered coordinates saved to D:\MD_file\Corning\las2o6\1012_cube\lasp_inp.region


In [82]:

ele_dic = {'O':1,'Si':2,'Al':3,'Li':4}
# 将坐标写回到xyz文件中
def write_xyz_file(filename, coordinates):
    with open(filename, 'w') as f:
        f.write(f"{len(coordinates)}\n")
        f.write("Atoms\n")
        for atom, x, y, z in coordinates:
            f.write(f"{ele_dic[atom]}    {x:.6f}    {y:.6f}    {z:.6f}\n")

# 使用示例
save_path = save_path  # 设置保存文件的路径和文件名
write_xyz_file(save_path, new_cry_xyz)
print(f"Filtered coordinates saved to {save_path}")


Filtered coordinates saved to D:\Atomsk\las_cif\las2o6_cube.xyz
